In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

In [2]:
df = pd.read_excel('data.xlsx', header=None)
requirements = df.values.flatten()
requirements

array([ 11,   5,   4,   7,  16,   6,   5,   7,  13,   6,   5,   7,  12,
         5,   4,   6,   9,   5,   5,  11,  29,  21,  17,  20,  27,  13,
         9,  10,  16,   6,   5,   7,  11,   5,   5,   6,  12,   7,   7,
        10,  15,  10,   9,  11,  15,  10,  10,  16,  26,  21,  23,  36,
        50,  45,  45,  49,  57,  43,  40,  44,  52,  43,  42,  45,  52,
        41,  39,  41,  48,  35,  34,  35,  42,  34,  36,  43,  55,  48,
        54,  65,  80,  70,  74,  85, 101,  89,  88,  90, 100,  87,  88,
        89, 104,  89,  89,  90, 106,  96,  94,  99, 109,  99,  96, 102],
      dtype=int64)

In [3]:
total_week = 104
waste_rate = 0.1
k = 20

In [4]:
hand_new = cp.Variable(total_week, integer=True)
hand_work = cp.Variable(total_week, integer=True)
hand_rest = cp.Variable(total_week, integer=True)
hand_teach = cp.Variable(total_week, integer=True)
hand_skill = cp.Variable(total_week, integer=True)
body_new = cp.Variable(total_week, integer=True)
body_work = cp.Variable(total_week, integer=True)
body_rest = cp.Variable(total_week, integer=True)
body_tested = cp.Variable(total_week, integer=True)
waste = cp.Variable(total_week, integer=True)

In [5]:
cons = [
    hand_work == 4 * requirements,
    body_work == requirements,
    k * hand_teach >= hand_new,
    hand_skill == hand_work + hand_rest + hand_teach,
    body_tested == body_work + body_rest,
    waste <= 0.2 * hand_work + 0.5,
    waste >= 0.2 * hand_work - 0.500001,

    hand_new >= 0,
    hand_rest >= 0,
    hand_work >= 0,
    hand_teach >= 0,
    body_new >= 0,
    body_rest >= 0,
    body_work >= 0,
    waste >= 0
]

In [6]:
for t in range(total_week - 1):
    cons.append(hand_rest[t+1] >= hand_work[t] - waste[t])
    cons.append(hand_skill[t+1] == hand_skill[t] + hand_new[t])
    cons.append(body_tested[t+1] == body_tested[t] + body_new[t])

In [7]:
obj = cp.Minimize(
    200 * cp.sum(body_new) + 
    100 * cp.sum(hand_new) + 
    5 * cp.sum(hand_rest) + 
    10 * cp.sum(body_rest) +
    10 * cp.sum(hand_new + hand_teach)
)

In [8]:
prob = cp.Problem(obj, cons)
prob.solve()

193750.0

In [9]:
hand_work.value[0:8], waste.value[0:8]

(array([44., 20., 16., 28., 64., 24., 20., 28.]),
 array([ 9.,  4.,  3.,  6., 13.,  5.,  4.,  6.]))